### Импорты

In [35]:
import sqlite3
import pandas as pd
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [36]:
from tqdm.auto import tqdm

In [37]:
import json

In [38]:
import re

In [39]:
con = sqlite3.connect('telegram.db')

In [40]:
c = con.cursor()
con.commit()

### Функции

Функция для лемматизации токена через PyMorphy.

In [103]:
def lemmatization(token):
    p = morph.parse(token)[0]
    return p.normal_form

Функция, которая, получая на вход токен, ищет точные совпадения.

In [47]:
def get_posts_for_token(qua):
    x = (qua, )
    
    list_id_posts = []
    for row in c.execute("""SELECT list_posts_id FROM tokens
                            WHERE token=?""", x):
        list_id_posts.extend(row[-1].replace('[', '').replace(']', '').replace(' ', '').split(','))
 
    channel_names = []
    post_texts = []
    for id_p in list_id_posts:
        z = (int(id_p), )
        for row in c.execute("""SELECT channels.name, posts.text FROM posts JOIN channels
            ON posts.id_channel = channels.id_channel WHERE id_post=?""", z):
            channel_names.append(row[-2])
            post_texts.append(row[-1])
    return post_texts, channel_names

Функция, которая, получая на вход токен, лемматизирует его, затем смотрит, какие у этой леммы есть токены, и выводит те посты, в которых эти токены есть.

In [48]:
def search_lemma(q):
    x = (q, )
    for row in c.execute("""SELECT id_token FROM tokens
        WHERE token=?""", x):
        i = str(row[-1])
    le = (lemmatization(q), )
    for row in c.execute("""SELECT list_tokens FROM lemmas
        WHERE lemma=?""", le):
        tokens_id = row[-1].replace('[', '').replace(']', '').replace(' ', '').split(',')
        tokens_id.remove(i)

    list_tokens = []
    for j in tokens_id:
        y = (j, )
        for row in c.execute("""SELECT token FROM tokens
                                WHERE id_token=?""", y):
            list_tokens.append(row[-1])

    post_texts = []
    channel_names = []
    for t in list_tokens:
        a, b = get_posts_for_token(t)
        post_texts.extend(a)
        channel_names.extend(b)
    return post_texts, channel_names

Функция, которая, получая на вход токен, смотрит, какие у него есть синонимы, и выводит те посты, в которых эти синонимы есть.

In [49]:
def search_synonyms(q):
    x = (q, )
    for row in c.execute("""SELECT list_synonyms FROM tokens
        WHERE token=?""", x):
        synonyms_id = row[-1]

    list_synonyms = []
    for j in synonyms_id:
        y = (j, )
        for row in c.execute("""SELECT token FROM tokens
                                WHERE id_token=?""", y):
            list_synonyms.append(row[-1])

    post_texts = []
    channel_names = []
    for s in list_synonyms:
        a, b = search_lemma(s)
        post_texts.extend(a)
        channel_names.extend(b)
    return post_texts, channel_names

Функция, которая выполняет поиск и сохраняет результат для одного токена.

In [94]:
def search_one(qua: str):
    q = qua.lower()
    post_texts, channel_names = get_posts_for_token(q)  
    a, b = search_lemma(q)
    post_texts.extend(a)
    channel_names.extend(b)
    c, d = search_synonyms(q)
    post_texts.extend(c)
    channel_names.extend(d)
    if len(post_texts) == 0:
        with open('posts.json', 'w', encoding='utf-8') as file:
            json.dump('No posts yet...', file)
    else: 
        df_output = pd.DataFrame()
        df_output['post'] = post_texts
        df_output['channel'] = channel_names
        df_output = df_output.drop_duplicates()
        output = df_output.to_dict('records')
        with open('posts.json', 'w', encoding='utf-8') as file:
            json.dump(output, file)

Функция, которая ищет точные вхождения более, чем одного токена в тексты (для этого был специально создан столбец text_clear, в котором тексты очищены от пунктуации и приведены к нижнему регистру).

In [62]:
def search_many_tokens(qua):
    channel_ids = []
    post_texts = []
    for row in c.execute("""SELECT text, id_channel, text_clear FROM posts"""):
        if qua in row[-1]:
            post_texts.append(row[-3])
            channel_ids.append(row[-2])
 
    channel_names = []
    for id_c in channel_ids:
        z = (int(id_c), )
        for row in c.execute("""SELECT channels.name FROM channels
            WHERE channels_id=?""", z):
            channel_names.append(row[-1])
    return post_texts, channel_names

Функция, которая лемматизирует запрос, а потом ищет точные вхождения лемматизириванного запроса в тексты (для этого был специально создан столбец text_lem, в котором тексты очищены от пунктуации, приведены к нижнему регистру и лемматизированы).

In [63]:
def search_many_lemmas(qua):
    qu = qua.split()
    lem_qua_l = [lemmatization(q) for q in qu]
    lem_qua = ' '.join(lem_qua_l)
    channel_ids = []
    post_texts = []
    for row in c.execute("""SELECT text, id_channel, text_lem FROM posts"""):
        if lem_qua in row[-1]:
            post_texts.append(row[-3])
            channel_ids.append(row[-2])
 
    channel_names = []
    for id_c in channel_ids:
        z = (int(id_c), )
        for row in c.execute("""SELECT channels.name FROM channels
            WHERE channels_id=?""", z):
            channel_names.append(row[-1])
    return post_texts, channel_names

Функция, которая выполняет поиск и сохраняет результат для нескольких токенов.

In [93]:
def search_many(qua):
    post_texts, channel_names = search_many_tokens(qua)
    a, b = search_many_lemmas(qua)
    post_texts.extend(a)
    channel_names.extend(b)
    if len(post_texts) == 0:
        with open('posts.json', 'w', encoding='utf-8') as file:
            json.dump('No posts yet...', file)
    else: 
        df_output = pd.DataFrame()
        df_output['post'] = post_texts
        df_output['channel'] = channel_names
        df_output = df_output.drop_duplicates()
        output = df_output.to_dict('records')
        with open('posts.json', 'w', encoding='utf-8') as file:
            json.dump(output, file)

Функция, выполняющая поиск для любого запроса.

In [65]:
def search(qua):
    qua = str(qua).lower()
    qua_l = qua.split()
    if len(qua)==0:
        with open('posts.json', 'w', encoding='utf-8') as file:
            json.dump('Your query is empty :(', file)
    elif len(qua)==1:
        search_one(qua)
    else:
        search_many(qua)

### Тестируем

In [95]:
search('тевирп')

In [96]:
with open('posts.json', 'r', encoding='utf-8') as f:
    text = json.load(f)
    print(text)

No posts yet...


In [97]:
search('')

In [98]:
with open('posts.json', 'r', encoding='utf-8') as f:
    text = json.load(f)
    print(text)

Your query is empty :(


In [99]:
search('телевизор')

In [100]:
with open('posts.json', 'r', encoding='utf-8') as f:
    text = json.load(f)
    print(text)

[{'post': 'Переносная приставка к телевизору Квант, выпускаемая на Львовском заводе кинескопов? 1974г', 'channel': 'che_history1'}, {'post': 'Начались длинные выходные. «Майские праздники» — это всегда своего рода водораздел между зимне-весенним и летне-осенним сезонами. Жизнью в квартире и жизнью вне дома (на даче, в отпуске, в парке и т.д.). Жизнь в квартире — это жизнь у экрана; на воздухе — это когда каждый день видишь окружающее своими глазами. И делаешь выводы. В выборный год это момент поворота от «политики из телевизора» к тому, что происходит вокруг нас в повседневной действительности. И поневоле человек переключается с вопросов из виртуального мира — «как бы нам показать и доказать Америке...» — на совсем другие вопросы: «как я живу?», «на что я живу?», «что происходит вокруг меня?»Именно поэтому мы пишем на баннерах партии «Новые люди» фразу «Люди важнее». Этим мы хотим сказать, что главное не то, кому и что доказала или доказывает наша страна на мировой арене. А то, что про

In [101]:
search('приставка к телевизору квант')

In [102]:
with open('posts.json', 'r', encoding='utf-8') as f:
    text = json.load(f)
    print(text)

[{'post': 'Переносная приставка к телевизору Квант, выпускаемая на Львовском заводе кинескопов? 1974г', 'channel': 'che_history1'}]


### Здесь добавляются столбцы в бд

Добавляю столбец с текстами постов, приведёнными к нижнему регистру и очищенными от знаков препинания.

In [29]:
all_texts = []
posts_ids = []
for row in c.execute('''SELECT posts_id, text FROM posts'''):
    all_texts.append(row[-1])
    posts_ids.append(row[-2])
clear_texts = []
patterns = """[!#$%&'()*+,./:;"«»<=>?@[\]^_`{|}~—\"\-]+"""
for t in all_texts:
    t1 = re.sub(patterns, ' ', t)
    t2 = t1.lower()
    clear_texts.append(t2)
t_p_c = [(posts_ids[i], clear_texts[i]) for i in range(len(all_texts))]
#c.execute('''ALTER TABLE posts ADD COLUMN text_clear TEXT''')
#con.commit()
for t in range(len(clear_texts)):
    c.execute('''UPDATE posts SET text_clear=?
        WHERE posts_id = ?''', (t_p_c[t][1], t_p_c[t][0]))

con.commit()

Добавляю столбец с лемматизированными текстами постов, приведёнными к нижнему регистру и очищенными от знаков препинания.

In [42]:
lem_texts = []
for t in tqdm(clear_texts):
    lem_t_l = [lemmatization(toke) for toke in t.split()]
    lem_t = ' '.join(lem_t_l)
    lem_texts.append(lem_t)
p_l = [(posts_ids[i], lem_texts[i]) for i in range(len(all_texts))]
#c.execute('''ALTER TABLE posts ADD COLUMN text_lem TEXT''')
con.commit()
for t in tqdm(range(len(all_texts))):
    c.execute('''UPDATE posts SET text_lem=?
        WHERE posts_id = ?''', (p_l[t][1], p_l[t][0]))
con.commit()

In [30]:
for t in range(len(clear_texts)):
    c.execute('''UPDATE posts SET text_clear=?
        WHERE posts_id = ?''', (t_p_c[t][1].replace('  ', ' '), t_p_c[t][0]))

con.commit()